### Imports

In [1]:
from itertools import groupby
from src import objects

### Define Sample Objects

#### Instantiate People

In [2]:
vin = objects.Person(name='Vin', role='Manager, Research')
will = objects.Person(name='Will', role='Researcher')
blake = objects.Person(name='Blake', role='Researcher')
sophia = objects.Person(name='Sophia', role='Researcher')
natalie = objects.Person(name='Natalie', role='Researcher')
chintan = objects.Person(name='Chintan', role='Researcher')
alex = objects.Person(name='Alex', role='Researcher')

#### Add People to a Team

In [3]:
research = objects.Team(name='Research', description='Dots Research Team')
research.members.append(vin)
research.members.append(will)
research.members.append(blake)
research.members.append(sophia)
research.members.append(natalie)
research.members.append(chintan)
research.members.append(alex)
research.members

7 members of type Person

#### Define Attribute Taxonomies

In [4]:
composure = objects.Attribute(name='Composure', description='Keeping your cool')
synthesis = objects.Attribute(name='Synthesis')
synth_situ = objects.Attribute(name='Synthesizing the Situation')
synth_time = objects.Attribute(name='Synthesizing through Time')
synthesis.members.append(synth_situ)
synthesis.members.append(synth_time)

In [5]:
synthesis.members

2 members of type Attribute

In [6]:
for item in synthesis.members:
    print(item.name)

Synthesizing the Situation
Synthesizing through Time


#### Define a Meeting

In [7]:
weekly_dave_sync = objects.Meeting(name='Weekly Dave Sync', description='Present progress to Dave')
question = objects.Question(
    name='Status Check', 
    description='Is the meeting going as expected?', 
    question_type=objects.QuestionType.BINARY
)

#### Add Participants

In [8]:
for person in research.members:
    weekly_dave_sync.participants.append(person)

#### Record Responses to a Question

In [9]:
response1 = objects.Response(source=vin, target=question, value=True, context=[research, weekly_dave_sync])
response2 = objects.Response(source=will, target=question, value=True, context=[research, weekly_dave_sync])
response3 = objects.Response(source=alex, target=question, value=False, context=[research, weekly_dave_sync])

In [10]:
question.add_responses(response1)
question.add_responses(response2)
question.add_responses(response3)

In [11]:
weekly_dave_sync.questions.append(question)

#### Give Feedback in Meeting

In [12]:
dot1 = objects.Dot(source=vin, target=blake, measure=composure, value=5, context=[research, weekly_dave_sync])
dot2 = objects.Dot(source=vin, target=blake, measure=synth_situ, value=7, context=[research, weekly_dave_sync])
dot3 = objects.Dot(source=vin, target=blake, measure=synth_time, value=8, context=[research, weekly_dave_sync])
dot4 = objects.Dot(source=will, target=vin, measure=composure, value=4, context=[research, weekly_dave_sync])
dot5 = objects.Dot(source=will, target=vin, measure=synthesis, value=7, context=[research, weekly_dave_sync])
dot6 = objects.Dot(source=blake, target=chintan, measure=synth_situ, value=7, context=[research, weekly_dave_sync])
dot7 = objects.Dot(source=blake, target=chintan, measure=composure, value=4, context=[research, weekly_dave_sync])

In [13]:
weekly_dave_sync.dots.append(dot1)
weekly_dave_sync.dots.append(dot2)
weekly_dave_sync.dots.append(dot3)
weekly_dave_sync.dots.append(dot4)
weekly_dave_sync.dots.append(dot5)
weekly_dave_sync.dots.append(dot6)
weekly_dave_sync.dots.append(dot7)

#### Synthesize What Happened

In [14]:
weekly_dave_sync.participants

7 members of type Person

In [15]:
weekly_dave_sync.dots

7 members of type Dot

In [16]:
weekly_dave_sync.questions

1 members of type Question

In [17]:
for question in weekly_dave_sync.questions:
    print(question.responses)

3 members of type Response


#### Generate Logic by using Domain Objects

In [18]:
def select_dots_from_meeting(original_function):
    """This decorator selects dots from the Meeting object and
    passes it into the decorated function"""
    def new_function(meeting, *args,**kwargs):
        dots = meeting.dots.data
        return original_function(dots, *args,**kwargs)
    return new_function

In [19]:
@select_dots_from_meeting
def primary_participants_in_meeting_138(dots: objects.Meeting):
    """
    OUTPUT: AssertionSet
    INPUT: Dots
    CONTEXT: Meeting
    INSIGHT: Activity
    PROCESSING: Comparing [Person vs. Overall]

    * Identifies whether a Person received more than 20% of Dots (or more than 10% of Dots if they receive at least 10 Dots) during a Meeting
    * Returns a boolean representing whether a Person is a Primary Participant in a Meeting.
    * This is produced by the following operation(s):
        * Determines whether a Person is a Primary Participant in a Meeting if either of the following conditions are true:
         * Determines whether a Person received more than 20% of Dots in a Meeting
         * Determines whether a Person received more than 10% of Dots and more than 10 Dots in a Meeting
    """
    total_participation = len(dots)
    dots.sort(key=lambda x: x.source.name, reverse=False)
    for person, dots in groupby(dots, lambda x: x.target):
        person_dots = len(list(dots))
        if ((person_dots / total_participation) > 0.2) or (person_dots > 10 and ((person_dots / total_participation) > 0.1)):
            yield objects.Judgement(source='System', target=person, value=True)
        else:
            yield objects.Judgement(source='System', target=person, value=False)

In [20]:
results = primary_participants_in_meeting_138(weekly_dave_sync)

In [21]:
for result in results:
    print(result.source, result.target.name, result.value)

System Chintan True
System Blake True
System Vin True


In [22]:
2/7

0.2857142857142857